### Recomendador de películas Disney

In [1]:
# se importan las librerías necesarias
import pandas as pd
from IPython.display import display
import re

##### Este recomendador sigue la estructura de una ETL

In [2]:
def extract():
    df_given = pd.read_csv('disney_movies.csv') # se extraen los datos del csv en forma de dataframe
    return df_given

In [3]:
def transform(df_given):
    total_missings = df_given.isnull().sum() # se obtiene el total de valores nulos
    columns_df = df_given.columns.tolist() # se saca una lista con los nombres de las columnas
    df = df_given
    # print(total_missings != 0) # devuelve True para 'genre' y para 'mpaa_rating' (en estas categorías hay Nones)
    for column in columns_df:
        if total_missings[column] != 0: # se cogen las columnas en las que haya valores nulos
            df[column] = df_given[column].fillna('No ' + column) # se reemplaza cada valor nulo por un 'no existe' dependiendo de la columna
    
    genres = df['genre'].unique().tolist() # se saca una lista con todos los posibles géneros
    genres.remove('No genre') # se elimina la opción de 'género no definido' de los posibles géneros
    df_nogenre = df[df['genre'] == 'No genre'] # se obtiene el dataframe que contiene todas las películas con 'género no definido'
    df_nogenre = df_nogenre.sort_values(by = 'total_gross', ascending = False).reset_index() 
    # se ordena el dataframe de películas sin género en base al éxito que tuvieron (dinero ganado)
    
    selection = ''
    while selection not in genres: # el input introducido con el género deseado ha de ser idéntico al género incluido en la lista de opciones
        selection = input('Select a movie genre out of the following: ' + str(genres) + ' ')
    
    df_search = pd.DataFrame(columns = columns_df) # se crea un nuevo dataframe con las mismas columnas que el dataframe para poder modificarlo
    regex_use = re.compile(selection, re.I) # se emplea regex ignorando las mayúsculas en la búsqueda
    for i in range(df.shape[0]): # se busca a lo largo de todas las filas del dataframe
        ocurrencia = re.findall(regex_use, df['genre'][i]) # se busca la selección con el método 'findall' de re
        if ocurrencia != []: # si 'findall' devuelve alguna ocurrencia
            df_search = df_search.append(df.iloc[i], ignore_index = True) # se añade la fila con el género seleccionado al dataframe de búsqueda
            # van a quitar la función append de versiones futuras de pandas
    df_search = df_search.sort_values(by = 'total_gross', ascending = False).reset_index() # se ordena el dataframe de búsqueda en base al éxito
    return df_search, df_nogenre

In [4]:
def load(df_search, df_nogenre):
    print('The first suggested films for you are the following: ')
    display(df_search.head(10)) # se ofrecen las 10 primeras recomendaciones de películas
    more = ''
    contador = 10
    while more not in ['n', 'N']: # se continúa preguntando por recomendaciones hasta que se especifique que no se quieren más
        more = ''
        while more not in ['y', 'n', 'Y', 'N']: # se continúa preguntando si la respuesta no es válida
            more = input('Would you like some more suggestions? [Y/N] ')
        if more == 'y' or more == 'Y': # si se indica que sí, se ofrecen 10 recomendaciones más
            if not df_search[contador:contador+10].empty:
                display(df_search[contador:contador+10])
                contador += 10 # se guarda que se han mostrado 10 recomendaciones más, para futuras recomendaciones
            else:
                print('There are no more available suggestions for the selected genre, but here are other films you may enjoy')
                # si no quedan recomendaciones del género seleccionado, se muestran las películas de género no definido
                display(df_nogenre)
                more = 'n'

In [5]:
if __name__ == '__main__':
    df_given = extract() # se sacan los datos del csv 
    df_search, df_nogenre = transform(df_given) # se transforman los datos para obtener los dataframes con los que queremos trabajar
    load(df_search, df_nogenre) # se muestran las recomendaciones pedidas empleando los dataframes creados

The first suggested films for you are the following: 


C:\Users\victo\AppData\Local\Temp\ipykernel_3568\308724299.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df.iloc[i], ignore_index = True) # se añade la fila con el género seleccionado al dataframe de búsqueda
C:\Users\victo\AppData\Local\Temp\ipykernel_3568\308724299.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df.iloc[i], ignore_index = True) # se añade la fila con el género seleccionado al dataframe de búsqueda
C:\Users\victo\AppData\Local\Temp\ipykernel_3568\308724299.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df.iloc[i], ignore_index = True) # se añade la fila con el género seleccionado al dataframe de búsqued

,index,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,3,An American Werewolf in Paris,1997-12-25,Horror,R,26570463,48546161
1,4,Stay Alive,2006-03-24,Horror,PG-13,23086480,29712825
2,5,Fright Night,2011-08-19,Horror,R,18298649,19452402
3,2,The Puppet Masters,1994-10-21,Horror,R,8579626,17727017
4,0,Something Wicked This Way Comes,1983-04-29,Horror,No mpaa_rating,5656087,15136765
5,1,Innocent Blood,1992-09-25,Horror,R,4877567,9907922


There are no more available suggestions for the selected genre, but here are other films you may enjoy


,index,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,23,The Black Hole,1979-12-21,No genre,No mpaa_rating,35841901,120377374
1,22,Herbie Goes to Monte Carlo,1977-06-24,No genre,No mpaa_rating,28000000,105847527
2,26,The Devil and Max Devlin,1981-01-01,No genre,No mpaa_rating,16000000,48517980
3,25,The Last Flight of Noah’s Ark,1980-06-25,No genre,No mpaa_rating,11000000,34472116
4,146,Bound by Honor,1993-04-16,No genre,R,4496583,9156084
5,122,Passed Away,1992-04-24,No genre,PG-13,4030793,8187848
6,128,A Gun in Betty Lou's Handbag,1992-08-21,No genre,PG-13,3591460,7295423
7,156,Father Hood,1993-08-27,No genre,PG-13,3268203,6654819
8,155,My Boyfriend's Back,1993-08-06,No genre,PG-13,3218882,6554384
9,24,Midnight Madness,1980-02-08,No genre,No mpaa_rating,2900000,9088096
